## Assignment 1
# Group 13
Mathieu Mailhot - Isabel Lougheed - Frank-Lucas Pantazis

Imports

In [5]:
import matplotlib
import numpy as np
import pandas as pd

In [2]:
"""
- CKD: 28 numerical features, 1 target binary classification variable ("Normal" / "CKD")
- Battery: 32 real-valued features, 2 classes ("Normal" / "Defective")
"""

# load data sets


# statistical analysis on the datasets

# - normalize

# models: all features, selective features based on statistical analysis (dropping features)

'\n- CKD: 28 numerical features, 1 target binary classification variable ("Normal" / "CKD")\n- Battery: 32 real-valued features, 2 classes ("Normal" / "Defective")\n'

Statistics

In [15]:
# load data sets
df_CKD = pd.read_csv("CKD.csv")
df_battery = pd.read_csv("Battery_Dataset.csv")

# Convert "CKD" to 1 and "Normal" to 0
df_CKD["label"] = df_CKD["label"].replace({"CKD": 1, "Normal": 0})
# Convert "Defective" to 1 and "Normal" to 0
df_battery["label"] = df_battery["label"].replace({"Defective": 1, "Normal": 0})

# Convert to a numpy array
CKD_data = df_CKD.to_numpy()
battery_data = df_battery.to_numpy()

print(CKD_data)
print(battery_data)


[[1.00000e+00 2.04500e-01 7.64600e-01 ... 8.81170e+01 9.47710e+01
  1.00000e+00]
 [2.00000e+00 4.82200e-01 3.53300e-01 ... 5.77680e+01 1.07200e+02
  1.00000e+00]
 [3.00000e+00 4.60200e-01 7.85100e-01 ... 1.05234e+02 9.16810e+01
  1.00000e+00]
 ...
 [3.28000e+02 2.53200e-01 6.52300e-01 ... 8.63860e+01 9.92350e+01
  1.00000e+00]
 [3.29000e+02 4.24900e-01 4.52800e-01 ... 1.20111e+02 9.83120e+01
  1.00000e+00]
 [3.30000e+02 5.40200e-01 3.66400e-01 ... 1.01204e+02 9.84400e+01
  1.00000e+00]]
[[1.000000e+00 5.854000e-01 6.264000e-01 ... 9.907300e+01 1.053899e+02
  1.000000e+00]
 [2.000000e+00 6.277000e-01 3.841000e-01 ... 9.150650e+01 9.100220e+01
  0.000000e+00]
 [3.000000e+00 5.428000e-01 2.937000e-01 ... 9.400780e+01 1.046438e+02
  0.000000e+00]
 ...
 [7.300000e+02 6.007000e-01 1.742000e-01 ... 1.117456e+02 9.444620e+01
  0.000000e+00]
 [7.310000e+02 5.074000e-01 4.975000e-01 ... 1.080145e+02 6.329760e+01
  1.000000e+00]
 [7.320000e+02 3.038000e-01 2.555000e-01 ... 1.227282e+02 7.608850e+

C:\Users\mmail\AppData\Local\Temp\ipykernel_10996\154256677.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_CKD["label"] = df_CKD["label"].replace({"CKD": 1, "Normal": 0})
C:\Users\mmail\AppData\Local\Temp\ipykernel_10996\154256677.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_battery["label"] = df_battery["label"].replace({"Defective": 1, "Normal": 0})
